##Libraries&Load Data


In [1]:
import pandas as pd
import numpy as np
import random
import tensorflow as tf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.arima_model import ARIMAResults
import joblib
import pickle
from scipy.optimize import minimize, LinearConstraint

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path = '/content/drive/MyDrive/Tesi/data/Dati/'
data_load_path = path + 'data/load/data_load.xlsx'
data_prod_path = path + 'data/production/data_production.xlsx'
data_load = pd.read_excel(data_load_path, decimal=',')
data_prod = pd.read_excel(data_prod_path, decimal=',')
data_load = data_load.set_index("datetime")
data_prod = data_prod.set_index("datetime")

In [4]:
def df_to_X_y(df, window_size):
  df_as_np = df.to_numpy()
  X = []
  y = []
  for i in range(len(df_as_np) - window_size):
      row = [a for a in df_as_np[i : i + window_size]]
      X.append(row)
      label = df_as_np[i + window_size]
      y.append(label)
  return np.array(X), np.array(y)

In [5]:
win_size = 3
X_load, y_load = df_to_X_y(data_load, win_size)
X_prod, y_prod = df_to_X_y(data_prod, win_size)

y_load = y_load.flatten()
y_prod = y_prod.flatten()

In [6]:
# Load the models

model_load_path = path + 'models/load/gru_model.keras'
model_load_gru = tf.keras.models.load_model(model_load_path)

In [7]:
model_prod_path = path + 'models/production/sarima_model.pkl'
from statsmodels.tsa.arima_model import ARIMAResults
model_prod_sarima = joblib.load(model_prod_path)

In [8]:
# Make predictions using the loaded model

load = data_load.iloc[3:]
load['pred'] = model_load_gru.predict(X_load).flatten()
load

274/274 [==============================] - 1s 2ms/step


<ipython-input-8-c483df6ed651>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  load['pred'] = model_load_gru.predict(X_load).flatten()


,carico,pred
datetime,,
2013-01-01 03:00:00,0.071443,0.117215
2013-01-01 04:00:00,0.071443,0.113994
2013-01-01 05:00:00,0.071443,0.114024
2013-01-01 06:00:00,0.071443,0.114461
2013-01-01 07:00:00,0.071443,0.114461
...,...,...
2013-12-31 19:00:00,0.295625,-0.308975
2013-12-31 20:00:00,0.295625,-0.440229
2013-12-31 21:00:00,0.285771,-0.156215


In [9]:
forecast_steps = 8757
prod = data_prod.iloc[3:]
df_prod = pd.DataFrame()
df_prod['pred'] = model_prod_sarima.forecast(steps=forecast_steps)
df_prod

,pred
2013-11-01 00:00:00,4.847620e-05
2013-11-01 01:00:00,8.287851e-05
2013-11-01 02:00:00,9.614792e-05
2013-11-01 03:00:00,9.660002e-05
2013-11-01 04:00:00,9.000880e-05
...,...
2014-10-31 16:00:00,1.539265e-02
2014-10-31 17:00:00,8.838468e-04
2014-10-31 18:00:00,2.650953e-05
2014-10-31 19:00:00,4.237132e-07


##Creation Data Structure

In [10]:
ESS_storage_max = 0.5
ESS_storage = 0
n_prosumers = 3
n_consumers = 3
prosumers = []
prosumers_c = []
consumers = []
energy_price = 0.174
energy_price_2 = 0.180
omega_t = [str(ts) for ts in load.index]

D_th = [[]]
C_th = [[]]

In [11]:
# Generation of Producers and Consumers

s_p = 'P'
for i in range(0, n_prosumers):
    r = s_p + str(i+1)
    prosumers.append(r)

s_pc = 'P_C'
for i in range(0, n_prosumers):
    r = s_pc + str(i+1)
    prosumers_c.append(r)

s_c = 'C'
for i in range(0, n_consumers):
    u = s_c + str(i+1)
    consumers.append(u)

prosumers_consumers = prosumers + prosumers_c + consumers + ['tot_energy']
suppliers = ['local', 'ESS', 'Grid', 'tot_load']
ESS = 0
Grid = 0

In [12]:
# Creation Dataframe M for time t (M[t])

zeros_array = np.zeros((len(suppliers), len(prosumers_consumers)))
M = pd.DataFrame(zeros_array, index=suppliers, columns=prosumers_consumers)
M

,P1,P2,P3,P_C1,P_C2,P_C3,C1,C2,C3,tot_energy
local,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ESS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Grid,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
tot_load,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


##Data Generation (x 8760 hours)

In [13]:
# Data Generation (1 hour) * 8760 (hour in a year)
#es. 2013-01-01 12:00:00

In [14]:
# Generation of data

tot_load_m = []
tot_prod_m = []

def create_M(t):
    p = prod.loc[t, 'produzionePV']
    l = load.loc[t, 'pred']
    tot_load = []
    tot_prod = []
    for i in range(n_prosumers):
        s_pi = s_p + str(i+1)
        # if there is production
        if (p > 0):
          # put energy in the ESS (value predicted + deviation)
          deviation = random.uniform(-0.02, 0.02)
          pi = p + deviation
          tot_prod.append(pi)
          tot_load.append(0)
          M[s_pi].loc['local'] = pi
        else:
          tot_load.append(0)
          tot_prod.append(0)

    for i in range(n_prosumers):
        s_pci = s_pc + str(i+1)
        deviation = random.uniform(-0.02, 0.02)
        ci = l + deviation
        tot_prod.append(0)
        tot_load.append(ci)
        M[s_pci].loc['local'] = ci

    for i in range(n_consumers):
        s_ci = s_c + str(i+1)
        deviation = random.uniform(-0.02, 0.02)
        ci = l + deviation
        tot_prod.append(0)
        tot_load.append(ci)
        M[s_ci].loc['local'] = ci

    tot_prod_m.append(tot_prod)
    tot_load_m.append(tot_load)
    M['tot_energy'].loc['ESS'] = ESS_storage
    M['tot_energy'].loc['Grid'] = sum(M.loc['Grid'].iloc[:-1])
    return M

In [15]:
dt_M = {}

for t in range(len(omega_t)):
    dt_M[omega_t[t]] = create_M(omega_t[t])

In [16]:
# Save to Excel file
#dt_M.to_excel(index=True)

In [17]:
dt_M

{'2013-01-01 03:00:00':                 P1        P2        P3      P_C1      P_C2      P_C3  \
 local     0.056858  0.058064  0.058527  0.058394  0.088575  0.070997   
 ESS       0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
 Grid      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
 tot_load  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
 
                 C1        C2        C3  tot_energy  
 local     0.082735  0.084841  0.090054         0.0  
 ESS       0.000000  0.000000  0.000000         0.0  
 Grid      0.000000  0.000000  0.000000         0.0  
 tot_load  0.000000  0.000000  0.000000         0.0  ,
 '2013-01-01 04:00:00':                 P1        P2        P3      P_C1      P_C2      P_C3  \
 local     0.056858  0.058064  0.058527  0.058394  0.088575  0.070997   
 ESS       0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
 Grid      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
 tot_load  0.000000  0.0

# Optimization problem


## Local energy cost < External grid energy cost

In [112]:
energy_price_local = 0.174
energy_price_grid = 0.2

In [113]:
omega_t = [str(ts) for ts in load.index]                                        # Intervalli t
omega_alpha = prosumers + prosumers_c + consumers                               # Agenti
                                                                                # [[0] * len(omega_alpha) for _ in range(len(omega_t))]

G_th = [[0] * len(omega_alpha) for _ in range(len(omega_t))]                                                                        # Energia importata dal Grid da h in t
E_th = [[0] * len(omega_alpha) for _ in range(len(omega_t))]                                                                        # Energia esportata nel Grid da h in t

X_p2p_th = [[0] * len(omega_alpha) for _ in range(len(omega_t))]                                                                    # Energia esportata nell'ESS da h in t
I_p2p_th = [[0] * len(omega_alpha) for _ in range(len(omega_t))]                                                                    # Energia importata dall'ESS da h in t

D_th = [[4] * len(omega_alpha) for _ in range(len(omega_t))]                    # potenza di scarica dell’unità di accumulo connessa ad h in t
C_th = [[4] * len(omega_alpha) for _ in range(len(omega_t))]                    # potenza di carica dell’unità di accumulo connessa ad h in t

dem_th = tot_load_m.copy()                                                      # domanda complessiva di consumo elettrico di h in t
res_pv_th = tot_prod_m.copy()                                                   # produzione complessiva delle RES connesse ad h in t

C_spot = [energy_price_local    for _ in range(len(omega_t))]                               # prezzo orario spot [€/kWh] del mercato locale in t
C_LEM  = [energy_price_local    for _ in range(len(omega_t))]                               # prezzo orario [€/kWh] del mercato locale in t
C_sell = [energy_price_grid  for _ in range(len(omega_t))]                                  # prezzo di vendita orario del kWh in rete [€/kWh] in t.

In [114]:
t = 0
h = 0

def objective_function(x, t, h):
    G_th[t][h], E_th[t][h], X_p2p_th[t][h], I_p2p_th[t][h] = x
    return C_spot[t] * G_th[t][h] + C_LEM[t] * I_p2p_th[t][h] - C_sell[t] * E_th[t][h] - C_LEM[t] * X_p2p_th[t][h]

# Constraints
def constraint_1(x, t, h):
    G_th[t][h], E_th[t][h], X_p2p_th[t][h], I_p2p_th[t][h] = x
    return X_p2p_th[t][h] - I_p2p_th[t][h]

def constraint_2(x, t, h):
    G_th[t][h], E_th[t][h], X_p2p_th[t][h], I_p2p_th[t][h] = x
    lhs = G_th[t][h] + I_p2p_th[t][h] + D_th[t][h] + res_pv_th[t][h]
    rhs = dem_th[t][h] + X_p2p_th[t][h] + C_th[t][h] + E_th[t][h]
    return lhs - rhs

bounds = ((0, None),  # Bound for G
          (0, None),  # Bound for Ip2p
          (0, None),  # Bound for E
          (0, None))  # Bound for Xp2p

initial_guess = [0, 0, 0, 0,]
constraints = [
               {'type': 'ineq', 'fun': constraint_1, 'args': (t, h)},
               {'type': 'eq', 'fun': constraint_2, 'args': (t, h)}
             ]

result = 0
results_1 = [[0] * len(omega_alpha) for _ in range(len(omega_t))]
for h in range(len(omega_alpha)):
    for t in range(len(omega_t)):
        initial_guess = [0, 0, 0, 0,]
        result = minimize(objective_function, initial_guess, bounds=bounds, args=(t, h),constraints=constraints)
        results_1[t][h] = result
# fun : The value of the objective function at the optimal solution
# x : The optimal values of the variables
# jac : The Jacobian matrix of the constraints at the optimal solution. This indicates the sensitivity of the constraints with respect to changes in the variables.

In [115]:
import csv

column_titles = ['fun', 'x', 'jac']
with open('results_1.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(column_titles)
    for i in range(len(omega_t)):
        writer.writerow(["{"])
        for j in range(len(omega_alpha)):
            row_data = [results_1[i][j].fun, results_1[i][j].x, results_1[i][j].jac]
            row_str = ", ".join(map(str, row_data))
            writer.writerow(["[" + row_str + "]"])
        writer.writerow(["}"])

## Local energy cost > External grid energy cost

In [116]:
energy_price_local = 0.2
energy_price_grid = 0.174

In [117]:
omega_t = [str(ts) for ts in load.index]                                        # Intervalli t
omega_alpha = prosumers + prosumers_c + consumers                               # Agenti
                                                                                # [[0] * len(omega_alpha) for _ in range(len(omega_t))]

G_th = [[0] * len(omega_alpha) for _ in range(len(omega_t))]                                                                        # Energia importata dal Grid da h in t
E_th = [[0] * len(omega_alpha) for _ in range(len(omega_t))]                                                                        # Energia esportata nel Grid da h in t

X_p2p_th = [[0] * len(omega_alpha) for _ in range(len(omega_t))]                                                                    # Energia esportata nell'ESS da h in t
I_p2p_th = [[0] * len(omega_alpha) for _ in range(len(omega_t))]                                                                    # Energia importata dall'ESS da h in t

D_th = [[4] * len(omega_alpha) for _ in range(len(omega_t))]                    # potenza di scarica dell’unità di accumulo connessa ad h in t
C_th = [[4] * len(omega_alpha) for _ in range(len(omega_t))]                    # potenza di carica dell’unità di accumulo connessa ad h in t

dem_th = tot_load_m.copy()                                                      # domanda complessiva di consumo elettrico di h in t
res_pv_th = tot_prod_m.copy()                                                   # produzione complessiva delle RES connesse ad h in t

C_spot = [energy_price_local    for _ in range(len(omega_t))]                               # prezzo orario spot [€/kWh] del mercato locale in t
C_LEM  = [energy_price_local    for _ in range(len(omega_t))]                               # prezzo orario [€/kWh] del mercato locale in t
C_sell = [energy_price_grid     for _ in range(len(omega_t))]                               # prezzo di vendita orario del kWh in rete [€/kWh] in t

In [118]:
t = 0
h = 0

def objective_function(x, t, h):
    G_th[t][h], E_th[t][h], X_p2p_th[t][h], I_p2p_th[t][h] = x
    return C_spot[t] * G_th[t][h] + C_LEM[t] * I_p2p_th[t][h] - C_sell[t] * E_th[t][h] - C_LEM[t] * X_p2p_th[t][h]

# Constraints
def constraint_1(x, t, h):
    G_th[t][h], E_th[t][h], X_p2p_th[t][h], I_p2p_th[t][h] = x
    return X_p2p_th[t][h] - I_p2p_th[t][h]

def constraint_2(x, t, h):
    G_th[t][h], E_th[t][h], X_p2p_th[t][h], I_p2p_th[t][h] = x
    lhs = G_th[t][h] + I_p2p_th[t][h] + D_th[t][h] + res_pv_th[t][h]
    rhs = dem_th[t][h] + X_p2p_th[t][h] + C_th[t][h] + E_th[t][h]
    return lhs - rhs

bounds = ((0, None),  # Bound for G
          (0, None),  # Bound for Ip2p
          (0, None),  # Bound for E
          (0, None))  # Bound for Xp2p


constraints = [
               {'type': 'ineq', 'fun': constraint_1, 'args': (t, h)},
               {'type': 'eq', 'fun': constraint_2, 'args': (t, h)}
             ]

result = 0
results_2 = [[0] * len(omega_alpha) for _ in range(len(omega_t))]
for h in range(len(omega_alpha)):
    for t in range(len(omega_t)):
        initial_guess = [0, 0, 0, 0]
        result = minimize(objective_function, initial_guess, bounds=bounds, args=(t, h),constraints=constraints)
        results_2[t][h] = result
# fun : The value of the objective function at the optimal solution
# x : The optimal values of the variables
# jac : The Jacobian matrix of the constraints at the optimal solution. This indicates the sensitivity of the constraints with respect to changes in the variables.

In [119]:
import csv

column_titles = ['fun', 'x', 'jac']
with open('results_2.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(column_titles)
    for i in range(len(omega_t)):
        writer.writerow(["{"])
        for j in range(len(omega_alpha)):
            row_data = [results_2[i][j].fun, results_2[i][j].x, results_2[i][j].jac]
            row_str = ", ".join(map(str, row_data))
            writer.writerow(["[" + row_str + "]"])
        writer.writerow(["}"])

# Tests


In [ ]:
print(len(omega_t))
print(len(omega_alpha))
print(len(G_th))
print(len(G_th[0]))
print(len(E_th))
print(len(E_th[0]))
print(len(X_p2p_th))
print(len(X_p2p_th[0]))
print(len(I_p2p_th))
print(len(I_p2p_th[0]))
print(len(D_th))
print(len(D_th[0]))
print(len(C_th))
print(len(C_th[0]))
print(len(dem_th))
print(len(dem_th[0]))
print(len(res_pv_th))
print(len(res_pv_th[0]))
print(len(C_spot))
print(len(C_LEM))
print(len(C_sell))

In [18]:
tot_load_m[10][4]

0.030179775881300164

In [19]:
tot_prod_m[10][0]

0.23567272831246971

In [20]:
energy_price_local = 0.174
energy_price_grid = 0.2
G_th = 0
E_th = 0
X_p2p_th = 0
I_p2p_th = 0
D_th = 4
C_th = 4
# produzione
res_pv_th = 3
# consumo
dem_th = 2

C_spot = 0.174
C_LEM = 0.80
C_sell = 0.184
x = (G_th, E_th, X_p2p_th, I_p2p_th,)

In [21]:
def objective_function(x):
    G_th, E_th, X_p2p_th, I_p2p_th = x
    return C_spot * G_th + C_LEM * I_p2p_th - C_sell * E_th - C_LEM * X_p2p_th
def constraint_1(x):
    G_th, E_th, X_p2p_th, I_p2p_th = x
    return X_p2p_th - I_p2p_th
def constraint_2(x):
    G_th, E_th, X_p2p_th, I_p2p_th = x
    lhs = G_th + I_p2p_th + D_th + res_pv_th
    rhs = dem_th + X_p2p_th + C_th + E_th
    t = lhs - rhs
    return t
def constraint_3(x):
    G_th, E_th, X_p2p_th, I_p2p_th = x
    return dem_th - I_p2p_th + E_th
def constraint_4(x):
    G_th, E_th, X_p2p_th, I_p2p_th = x
    return res_pv_th - X_p2p_th + G_th

bounds = ((0, None), (0, None), (0, None), (0, None))
constraints = [{'type': 'eq', 'fun': constraint_1}, {'type': 'ineq', 'fun': constraint_2}, {'type': 'eq', 'fun': constraint_3}, {'type': 'eq', 'fun': constraint_4}]
initial_guess = [0, 0, 0, 0]
result = minimize(objective_function, initial_guess, bounds=bounds, constraints=constraints)
print(result)
G_th = result.x[0]
E_th = result.x[1]
X_p2p_th = result.x[2]
I_p2p_th = result.x[3]

res_pv_th_obt = X_p2p_th + G_th
dem_th_obt = I_p2p_th + E_th
print('res: ' + str(res_pv_th) + '        res obtained: ' + str(res_pv_th_obt))
print('dem: ' + str(dem_th)    + '        dem obtained: '  +  str(dem_th_obt))

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: -0.18400000000000194
       x: [ 1.946e-13  1.000e+00  3.000e+00  3.000e+00]
     nit: 2
     jac: [ 1.740e-01 -1.840e-01 -8.000e-01  8.000e-01]
    nfev: 12
    njev: 2
res: 3        res obtained: 3.000000000000389
dem: 2        dem obtained: 4.000000000000389
